In [85]:
import requests

url = "https://raw.githubusercontent.com/daivikpurani/LiveLikeALocal/main/Data%20Scraping/output.txt"
response = requests.get(url)

if response.status_code == 200:
    raw_html = response.text
    print("✅ File fetched successfully!")
    print(raw_html[:1000])  # Preview first 1000 chars
else:
    print(f"❌ Failed to fetch file. Status code: {response.status_code}")


✅ File fetched successfully!
URL: https://sf.funcheap.com/
H2: Funcheap's San Francisco City Guide
H2: Upcoming Fun and Cheap Events
H2: Tuesday, March 18, 2025
H2: 
H2: 
H3: 
H3: 
H3: 
H3: 
H3: 
H3: 
H3: Most Popular Upcoming Events
H3: Sign Up For Funcheap Emails
H3: Most Popular City Guides
H3: 
H3: 
H3: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: 
P: Don't Miss Another Bay Area Festival
P: Be first to hear about SF's newest fairs & festivals, weekend events, news and exclusive freebies, deals & promo codes.
LI: MENU
LI: ALL EVENTS
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: FREE
LI: TODAY
LI: WEEKEND
LI: WIN TIX
LI: NEWSLETTER
LI: FREE MUSEUM DAYS
LI: ADD EVENT
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 
LI: 80,000 Free Tulips Are Coming to SF’s Union Square (2025)
LI: SF’s Biggest Rooftop “Holi” Festival of Colors (2025)
LI: Grand Opening of SF’s New 10,000 sqft “Museum of Failu

In [87]:
from bs4 import BeautifulSoup
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

def clean_and_chunk(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    text = soup.get_text()
    text = re.sub(r'\s+', ' ', text).strip()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=["\n\n", "\n", ".", "!", "?"]
    )
    return splitter.split_text(text)

chunks = clean_and_chunk(raw_html)
print(f"✅ Cleaned and split into {len(chunks)} chunks")



✅ Cleaned and split into 1176 chunks


In [89]:
import re

def fully_clean_chunk(chunk):
    seen_lines = set()
    cleaned_lines = []

    for line in chunk.splitlines():
        # Remove any header prefixes like H2:, LI:, P:, URL:
        line = re.sub(r'^\s*(H\d:|LI:|P:|URL:)\s*', '', line).strip()

        # Skip empty or trivial lines
        if not line or re.fullmatch(r'[.,\- ]*', line):
            continue

        # Skip duplicate lines
        if line not in seen_lines:
            seen_lines.add(line)
            cleaned_lines.append(line)

    # Combine cleaned lines into one paragraph
    return ' '.join(cleaned_lines)

# Apply the cleaner to all your chunks
fully_cleaned_chunks = [fully_clean_chunk(chunk) for chunk in chunks]

# Filter out too-short ones
final_chunks = [c for c in fully_cleaned_chunks if len(c.split()) > 20]

# Preview the final cleaned chunks
for i, chunk in enumerate(final_chunks[:5]):
    print(f"\n--- Final Cleaned Chunk {i+1} ---\n{chunk}")



--- Final Cleaned Chunk 1 ---
https://sf.funcheap.com/ H2: Funcheap's San Francisco City Guide H2: Upcoming Fun and Cheap Events H2: Tuesday, March 18, 2025 H2: H2: H3: H3: H3: H3: H3: H3: H3: Most Popular Upcoming Events H3: Sign Up For Funcheap Emails H3: Most Popular City Guides H3: H3: H3: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: P: Don't Miss Another Bay Area Festival P: Be first to hear about SF's newest fairs & festivals, weekend events, news and exclusive freebies, deals & promo codes

--- Final Cleaned Chunk 2 ---
. LI: MENU LI: ALL EVENTS LI: LI: LI: LI: LI: LI: LI: LI: FREE LI: TODAY LI: WEEKEND LI: WIN TIX LI: NEWSLETTER LI: FREE MUSEUM DAYS LI: ADD EVENT LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: 80,000 Free Tulips Are Coming to SF’s Union Square (2025) LI: SF’s Biggest Rooftop “Holi” Festival of Colors (2025) LI: Grand Opening of SF’s New 10,000 sqft “Museum of Failure” (Fisherman’s Wharf) LI: 2025 S

In [75]:
CATEGORY_KEYWORDS = {
    "food": ["food", "restaurant", "eats", "dine", "drink", "culinary", "tasting", "brunch", "chefs", "beer", "wine", "bites"],
    "music": ["concert", "band", "dj", "music", "jazz", "live", "performance", "orchestra", "acoustic", "beats", "gig"],
    "family": ["kids", "family", "children", "parents", "craft", "games", "puppet", "art", "storytime", "outdoor"],
    "festival": ["festival", "parade", "carnival", "celebration", "fair", "block party", "street fair"],
    "shopping": ["market", "vendor", "popup", "shopping", "handmade", "boutique", "craft fair", "retail"],
    "cultural": ["heritage", "tradition", "diwali", "chinese new year", "black history", "latino", "asian", "indigenous"],
    "wellness": ["yoga", "meditation", "wellness", "fitness", "hike", "run", "health"],
}


In [79]:

def tag_chunk(text):
    tags = set()
    text_lower = text.lower()

    for category, keywords in CATEGORY_KEYWORDS.items():
        for keyword in keywords:
            if keyword in text_lower:
                tags.add(category)
                break  # Avoid overcounting keywords in same category

    return list(tags) if tags else ["uncategorized"]


In [81]:

tagged_chunks = []

for chunk in final_chunks:
    tags = tag_chunk(chunk)
    tagged_chunks.append({
        "text": chunk,
        "tags": tags,
        "metadata": {
            "source": "sfchronicle.com",
            "scraped_date": "2025-05-05",
            "tags": tags
        }
    })


In [83]:
for i, doc in enumerate(tagged_chunks[:5]):
    print(f"\n--- Tagged Chunk {i+1} ---")
    print("Tags:", doc["tags"])
    print("Content:", doc["text"][:300] + "...")



--- Tagged Chunk 1 ---
Tags: ['festival']
Content: https://sf.funcheap.com/ H2: Funcheap's San Francisco City Guide H2: Upcoming Fun and Cheap Events H2: Tuesday, March 18, 2025 H2: H2: H3: H3: H3: H3: H3: H3: H3: Most Popular Upcoming Events H3: Sign Up For Funcheap Emails H3: Most Popular City Guides H3: H3: H3: P: P: P: P: P: P: P: P: P: P: P: P:...

--- Tagged Chunk 2 ---
Tags: ['festival', 'shopping', 'food', 'family']
Content: . LI: MENU LI: ALL EVENTS LI: LI: LI: LI: LI: LI: LI: LI: FREE LI: TODAY LI: WEEKEND LI: WIN TIX LI: NEWSLETTER LI: FREE MUSEUM DAYS LI: ADD EVENT LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: 80,000 Free Tulips Are Coming to SF’s Union Square (2025) LI: SF’s Biggest Roofto...

--- Tagged Chunk 3 ---
Tags: ['festival']
Content: . Patrick’s Day Celebrations 2025 LI: San Francisco April Festivals & Street Fairs 2025 LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI: LI